In [1]:
import calendar
from collections import Counter
import pandas as pd

data = pd.read_csv('https://lms.skillfactory.ru/assets/courseware/v1/1d68c628a08cd5521ae70e976577568b/asset-v1:Skillfactory+DST-12+11MAR2020+type@asset+block/movie_bd_v5.csv')

# Предобработка

In [2]:
answers = {}
data['profit'] = data['revenue'] - data['budget']
data['genres_array'] = data['genres'].str.split('|')
data['overview_len'] = data['overview'].str.split().str.len()
data['cast_array'] = data['cast'].str.split('|')
data['production_array'] = data['production_companies'].str.split('|')
data['release_date'] = data['release_date'].apply(pd.to_datetime)

def movie_title(row):
    clean_title = row['original_title'].to_string(index=False).strip(' ')
    clean_imdb_id = row['imdb_id'].to_string(index=False).strip(' ')
    return f'{clean_title} ({clean_imdb_id})'

# 1. У какого фильма из списка самый большой бюджет?

In [3]:
row = data[data['budget'] == data['budget'].max()][['imdb_id', 'original_title']]
answers['1'] = movie_title(row)
print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [4]:
row = data[data['runtime'] == data['runtime'].max()][['imdb_id', 'original_title']]
answers['2'] =  movie_title(row)
print(answers['2'])

Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [5]:
row = data[data['runtime'] == data['runtime'].min()][['imdb_id', 'original_title']]
answers['3'] = movie_title(row)
print(answers['3'])

Winnie the Pooh (tt1449283)


# 4. Какова средняя длительность фильмов?


In [6]:
answers['4'] = round(data['runtime'].mean())
print(answers['4'])

110


# 5. Каково медианное значение длительности фильмов? 

In [7]:
answers['5'] = round(data['runtime'].median())
print(answers['5'])

107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [8]:
row = data[data['profit'] == data['profit'].max()][['imdb_id', 'original_title', 'profit']]
answers['6'] = movie_title(row)
print(answers['6'])

Avatar (tt0499549)


# 7. Какой фильм самый убыточный? 

In [9]:
row = data[data['profit'] == data['profit'].min()][['imdb_id', 'original_title', 'profit']]
answers['7'] = movie_title(row)
print(answers['7'])

The Lone Ranger (tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [10]:
answers['8'] = data[data['profit'] > 0]['imdb_id'].count()
print(answers['8'])

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [11]:
max_profit = data[data['release_year'] == 2008]['profit'].max()
row = data[(data['release_year'] == 2008) & (data['profit'] == max_profit)][['imdb_id', 'original_title', 'profit']]
answers['9'] = movie_title(row)
print(answers['9'])

The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [12]:
min_profit = data[data['release_year'].between(2012, 2014)]['profit'].min()
row = data[(data['release_year'].between(2012, 2014)) & (data['profit'] == min_profit)]
answers['10'] = movie_title(row)
print(answers['10'])

The Lone Ranger (tt1210819)


# 11. Какого жанра фильмов больше всего?

In [13]:
genres = data.explode('genres_array').groupby('genres_array')['imdb_id'].count().sort_values(ascending=False)
answers['11'] = genres.index[0]
print(answers['11'])

Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [14]:
with_profit = data[data['profit'] > 0]
genres = with_profit.explode('genres_array').groupby('genres_array')['imdb_id'].count().sort_values(ascending=False)
answers['12'] = genres.index[0]
print(answers['12'])

Drama


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [15]:
answers['13'] = data.groupby('director').sum()['revenue'].sort_values(ascending=False).index[0]
print(answers['13'])

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [16]:
variants = ['Ridley Scott', 'Guy Ritchie', 'Robert Rodriguez', 'Quentin Tarantino', 'Tony Scott']

action_directors = data[data['director'].isin(variants)]

action_directors = action_directors[action_directors['genres'].str.contains('Action')] \
    .groupby('director')['imdb_id'] \
    .count() \
    .sort_values(ascending=False)

answers['14'] = action_directors.index[0]
print(answers['14'])

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [17]:
data_2012 = data[data['release_year'] == 2012]
revenue = data_2012.explode('cast_array').groupby('cast_array')['revenue'].sum().sort_values(ascending=False)
answers['15'] = revenue.index[0]
print(answers['15'])

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [18]:
cast_budget = Counter()

def count_cast_budget(row):
    for cast in row['cast_array']:
        cast_budget[cast] += 1

data[data['budget'] > data['budget'].median()].apply(count_cast_budget, axis='columns')
answers['16'], _ = cast_budget.most_common(1)[0]
print(answers['16'])

Matt Damon


In [19]:
high_budget = data[data['budget'] > data['budget'].median()]
cast_hist = high_budget.explode('cast_array').groupby('cast_array')['imdb_id'].count().sort_values(ascending=False)
answers['16'] = cast_hist.index[0]
print(answers['16'])

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [20]:
cage = data[data['cast'].str.contains('Nicolas Cage')]
genres = cage.explode('genres_array').groupby('genres_array')['imdb_id'].count().sort_values(ascending=False)
answers['17'] = genres.index[0]
print(answers['17'])

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [21]:
paramount = data[data['production_companies'].str.contains('Paramount Pictures')]
answers['18'] = movie_title(paramount.sort_values(by='profit').head(1))
print(answers['18'])

K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [22]:
answers['19'] = data.groupby('release_year')['revenue'].sum().sort_values(ascending=False).index[0]
print(answers['19'])

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [23]:
warner = data[data['production_companies'].str.contains('Warner Bros')]
answers['20'] = warner.groupby('release_year')['revenue'].sum().sort_values(ascending=False).index[0]
print(answers['20'])

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [24]:
answers['21'] = data.groupby(by=data['release_date'].dt.month)['imdb_id'].count().sort_values(ascending=False).index[0]
print(calendar.month_name[answers['21']])

September


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [25]:
answers['22'] = data[data['release_date'].dt.month.between(6,8)]['imdb_id'].count()
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [26]:
directors = data[data['release_date'].dt.month.isin([12,1,2])].groupby('director')['imdb_id'].count().sort_values(ascending=False)
answers['23'] = directors.index[0]
print(answers['23'])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [27]:
title_len = {}
def title_count(row):
    for p in row['production_array']:
        title_len.setdefault(p, [])
        title_len[p].append(len(row['original_title']))
        

data.apply(title_count, axis='columns')
result = []
for p in title_len:
    result.append( (sum(title_len[p]) / len(title_len[p]), p)  )
_, answers['24'] = sorted(result, reverse=True)[0]
print(answers['24'])

Four By Two Productions


In [28]:
with_len = data.explode('production_array')[['production_array', 'original_title']]
with_len['original_title_len'] = with_len['original_title'].apply(len)
answers['24'] = with_len.groupby('production_array')['original_title_len'] \
    .mean() \
    .sort_values(ascending=False) \
    .index[0]
print(answers['24'])

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [29]:
answers['25'] = data.explode('production_array') \
    .groupby('production_array')['overview_len'] \
    .mean() \
    .sort_values(ascending=False) \
    .index[0]
print(answers['25'])

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [30]:
variants = [
    ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
    ['BloodRayne', 'The Adventures of Rocky & Bullwinkle'],
    ['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down'],
    ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'],
    ['Upside Down', 'Inside Out', 'Iron Man'],
]
best_99 = data[data['vote_average'] > data['vote_average'].quantile(.99)]['original_title']
best_set = set(best_99.unique())
answers['26'] = 'unknown'

for variant in variants:
    variant_set = set(variant)
    if set(variant).issubset(best_set):
        answers['26'] = ', '.join(variant)
    break

print(answers['26'])

Inside Out, The Dark Knight, 12 Years a Slave


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [31]:
pairs = Counter()
def count_pairs(row):
    for i, cast in enumerate(row['cast_array']):
        for cast2 in row['cast_array'][i:]:
            if cast != cast2:
                pairs[', '.join(sorted([cast, cast2]))] += 1

data.apply(count_pairs, axis='columns')
answers['27'], _ = pairs.most_common(1)[0]
print(answers['27'])

Daniel Radcliffe, Rupert Grint


# Submission

In [32]:
display(answers)

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [33]:
# и убедиться что ни чего не пропустил)
print(len(answers))

27
